# Feature Extraction from Unstructured Ob Notes 

In [1]:
import os, sys
import pickle

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
import seaborn as sns

In [5]:
import matplotlib.pyplot as plt

In [6]:
from datetime import datetime

In [7]:
import glob 

In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
from IPython.core.display import display, HTML    
display(HTML("<style>.container {width:90% !important; }</style>"))
%matplotlib inline 
np.set_printoptions(precision=5, suppress=True) 

In [9]:
pd.set_option('display.expand_frame_repr', False)

/Volumes/abraha1/projects/PTB_phenotyping/scripts/rand_forest_ptb_classification/parse_ob_notes


In [10]:
# root_path="/dors/capra_lab/users/"
root_path="/Volumes/"

In [11]:
sys.path.append( os.path.join(root_path,'/abraha1/projects/PTB_phenotyping/scripts/rand_forest_ptb_classification'))
os.chdir("/Volumes/abraha1/projects/PTB_phenotyping/scripts/rand_forest_ptb_classification/")
from hyperparam_tune import initialize, create_held_out,validate_best_model
from train_test_rf import load_labels

/Volumes/abraha1/projects/PTB_phenotyping/scripts/rand_forest_ptb_classification/train_test_rf.py:22: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/abraha1/anaconda3/envs/verBio/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/abraha1/anaconda3/envs/verBio/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/abraha1/anaconda3/envs/verBio/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/abraha1/anaconda3/envs/verBio/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/abraha1

# Functions

In [12]:
def replace_empty(value_list): 
    if len(value_list) == 0: 
        new_value = None 
    else: 
        new_value = value_list[0]
        
        return new_value

In [20]:
def parse_concepts_txt(concepts_dir):
    rows_list = []
    
    file_list = glob.glob(concepts_dir+"/*.txt")
    
    for index, concept_file in enumerate(file_list): 
        print("on {} out of {}".format(index, len(file_list)))
        with open(concept_file, 'r') as fh: 
            grid = os.path.basename(concept_file).split("-")[0]        

            for line in fh: 
                if (line.find('assertion=present') > -1) and line.startswith('NamedEntity'): 
                    _type, _pos1, _pos2 = line.split("\t")[:3]


                    assertion =  [x.split("=")[1].rstrip()  for x in line.split("\t") if (x.find('assertion')>-1)] 
                    semantic  =  [x.split("=")[1].rstrip()  for x in line.split("\t") if (x.find('semantic')>-1)] 
                    cui  =  [x.split("=")[1].rstrip()  for x in line.split("\t") if (x.find('cui')>-1)] 
                    ne  =  [x.split("=")[1].rstrip()  for x in line.split("\t") if (x.find('ne')>-1)] 

                    rows_list.append({'grid': grid,
                                       'type':_type,
                                       'pos_in_txt1': _pos1, 
                                       'pos_in_txt2': _pos2,
                                       'assertion': replace_empty(assertion),
                                       'semantic': replace_empty(semantic),
                                       'cui': replace_empty(cui),
                                       'ne': replace_empty(ne)})

    concepts_df = pd.DataFrame(rows_list)

    return concepts_df 

# Load Data 

In [30]:
CONCEPTS_DIR=os.path.join(root_path, "abraha1/projects/PTB_phenotyping/data/ptb_predict_machine_learning/ob_notes_variables/unstructured/notes_txt_clamp/")
RAW_TXT_DIR=os.path.join(root_path, "abraha1/projects/PTB_phenotyping/data/ptb_predict_machine_learning/ob_notes_variables/unstructured/notes_txt/")
KEY_FILE=os.path.join(root_path, "abraha1/projects/PTB_phenotyping/data/ptb_predict_machine_learning/ob_notes_variables/unstructured/key_icd9_10_grids.txt_Attributes.LD-2.tsv")

DELIVERY_LABELS_FILE = os.path.join(root_path, "abraha1/projects/PTB_phenotyping/data/combined_cohorts_07_12_2018/full_dataset_characterization/est_delivery_date_at_least_one_icd_cpt_ega.tsv")
FEAT_OUTPUT_DIR =os.path.join(root_path, "abraha1/projects/PTB_phenotyping/data/ptb_predict_machine_learning/feature_matrices/ob_notes_unstruct")

## Load delivery labels

In [164]:
final_labels_df = load_labels(DELIVERY_LABELS_FILE)
final_labels_dict =  dict(zip(final_labels_df.GRID, final_labels_df.label))
final_dates_dict =  dict(zip(final_labels_df.GRID, final_labels_df.delivery_date))

Loading delivery labels...
	0 out of 39008
	5000 out of 39008
	10000 out of 39008
	15000 out of 39008
	20000 out of 39008
	25000 out of 39008
	30000 out of 39008
	35000 out of 39008
	Created delivery-type labels df with shape:36379,3


In [165]:
final_labels_df.head()

,GRID,delivery_date,label
0,R200000594,2006-04-05,preterm
1,R200006865,2015-12-19,term
2,R200009152,2007-04-26,postterm
3,R200009182,2014-07-13,term
4,R200009647,2014-01-15,preterm


In [166]:
# pickle.dump( final_labels_df, open( DELIVERY_LABELS_FILE +".pickle", "wb" ) )

In [167]:
# check for duplicate GRIDS
print("number of duplicated grids in delivery label df: {}".format(final_labels_df[final_labels_df.duplicated(subset="GRID")].shape[0]))

number of duplicated grids in delivery label df: 0


## Load concepts

In [23]:
concepts_df = parse_concepts_txt(CONCEPTS_DIR)

on 0 out of 2434
on 1 out of 2434
on 2 out of 2434
on 3 out of 2434
on 4 out of 2434
on 5 out of 2434
on 6 out of 2434
on 7 out of 2434
on 8 out of 2434
on 9 out of 2434
on 10 out of 2434
on 11 out of 2434
on 12 out of 2434
on 13 out of 2434
on 14 out of 2434
on 15 out of 2434
on 16 out of 2434
on 17 out of 2434
on 18 out of 2434
on 19 out of 2434
on 20 out of 2434
on 21 out of 2434
on 22 out of 2434
on 23 out of 2434
on 24 out of 2434
on 25 out of 2434
on 26 out of 2434
on 27 out of 2434
on 28 out of 2434
on 29 out of 2434
on 30 out of 2434
on 31 out of 2434
on 32 out of 2434
on 33 out of 2434
on 34 out of 2434
on 35 out of 2434
on 36 out of 2434
on 37 out of 2434
on 38 out of 2434
on 39 out of 2434
on 40 out of 2434
on 41 out of 2434
on 42 out of 2434
on 43 out of 2434
on 44 out of 2434
on 45 out of 2434
on 46 out of 2434
on 47 out of 2434
on 48 out of 2434
on 49 out of 2434
on 50 out of 2434
on 51 out of 2434
on 52 out of 2434
on 53 out of 2434
on 54 out of 2434
on 55 out of 2434
on

on 445 out of 2434
on 446 out of 2434
on 447 out of 2434
on 448 out of 2434
on 449 out of 2434
on 450 out of 2434
on 451 out of 2434
on 452 out of 2434
on 453 out of 2434
on 454 out of 2434
on 455 out of 2434
on 456 out of 2434
on 457 out of 2434
on 458 out of 2434
on 459 out of 2434
on 460 out of 2434
on 461 out of 2434
on 462 out of 2434
on 463 out of 2434
on 464 out of 2434
on 465 out of 2434
on 466 out of 2434
on 467 out of 2434
on 468 out of 2434
on 469 out of 2434
on 470 out of 2434
on 471 out of 2434
on 472 out of 2434
on 473 out of 2434
on 474 out of 2434
on 475 out of 2434
on 476 out of 2434
on 477 out of 2434
on 478 out of 2434
on 479 out of 2434
on 480 out of 2434
on 481 out of 2434
on 482 out of 2434
on 483 out of 2434
on 484 out of 2434
on 485 out of 2434
on 486 out of 2434
on 487 out of 2434
on 488 out of 2434
on 489 out of 2434
on 490 out of 2434
on 491 out of 2434
on 492 out of 2434
on 493 out of 2434
on 494 out of 2434
on 495 out of 2434
on 496 out of 2434
on 497 out o

on 877 out of 2434
on 878 out of 2434
on 879 out of 2434
on 880 out of 2434
on 881 out of 2434
on 882 out of 2434
on 883 out of 2434
on 884 out of 2434
on 885 out of 2434
on 886 out of 2434
on 887 out of 2434
on 888 out of 2434
on 889 out of 2434
on 890 out of 2434
on 891 out of 2434
on 892 out of 2434
on 893 out of 2434
on 894 out of 2434
on 895 out of 2434
on 896 out of 2434
on 897 out of 2434
on 898 out of 2434
on 899 out of 2434
on 900 out of 2434
on 901 out of 2434
on 902 out of 2434
on 903 out of 2434
on 904 out of 2434
on 905 out of 2434
on 906 out of 2434
on 907 out of 2434
on 908 out of 2434
on 909 out of 2434
on 910 out of 2434
on 911 out of 2434
on 912 out of 2434
on 913 out of 2434
on 914 out of 2434
on 915 out of 2434
on 916 out of 2434
on 917 out of 2434
on 918 out of 2434
on 919 out of 2434
on 920 out of 2434
on 921 out of 2434
on 922 out of 2434
on 923 out of 2434
on 924 out of 2434
on 925 out of 2434
on 926 out of 2434
on 927 out of 2434
on 928 out of 2434
on 929 out o

on 1297 out of 2434
on 1298 out of 2434
on 1299 out of 2434
on 1300 out of 2434
on 1301 out of 2434
on 1302 out of 2434
on 1303 out of 2434
on 1304 out of 2434
on 1305 out of 2434
on 1306 out of 2434
on 1307 out of 2434
on 1308 out of 2434
on 1309 out of 2434
on 1310 out of 2434
on 1311 out of 2434
on 1312 out of 2434
on 1313 out of 2434
on 1314 out of 2434
on 1315 out of 2434
on 1316 out of 2434
on 1317 out of 2434
on 1318 out of 2434
on 1319 out of 2434
on 1320 out of 2434
on 1321 out of 2434
on 1322 out of 2434
on 1323 out of 2434
on 1324 out of 2434
on 1325 out of 2434
on 1326 out of 2434
on 1327 out of 2434
on 1328 out of 2434
on 1329 out of 2434
on 1330 out of 2434
on 1331 out of 2434
on 1332 out of 2434
on 1333 out of 2434
on 1334 out of 2434
on 1335 out of 2434
on 1336 out of 2434
on 1337 out of 2434
on 1338 out of 2434
on 1339 out of 2434
on 1340 out of 2434
on 1341 out of 2434
on 1342 out of 2434
on 1343 out of 2434
on 1344 out of 2434
on 1345 out of 2434
on 1346 out of 2434


on 1716 out of 2434
on 1717 out of 2434
on 1718 out of 2434
on 1719 out of 2434
on 1720 out of 2434
on 1721 out of 2434
on 1722 out of 2434
on 1723 out of 2434
on 1724 out of 2434
on 1725 out of 2434
on 1726 out of 2434
on 1727 out of 2434
on 1728 out of 2434
on 1729 out of 2434
on 1730 out of 2434
on 1731 out of 2434
on 1732 out of 2434
on 1733 out of 2434
on 1734 out of 2434
on 1735 out of 2434
on 1736 out of 2434
on 1737 out of 2434
on 1738 out of 2434
on 1739 out of 2434
on 1740 out of 2434
on 1741 out of 2434
on 1742 out of 2434
on 1743 out of 2434
on 1744 out of 2434
on 1745 out of 2434
on 1746 out of 2434
on 1747 out of 2434
on 1748 out of 2434
on 1749 out of 2434
on 1750 out of 2434
on 1751 out of 2434
on 1752 out of 2434
on 1753 out of 2434
on 1754 out of 2434
on 1755 out of 2434
on 1756 out of 2434
on 1757 out of 2434
on 1758 out of 2434
on 1759 out of 2434
on 1760 out of 2434
on 1761 out of 2434
on 1762 out of 2434
on 1763 out of 2434
on 1764 out of 2434
on 1765 out of 2434


on 2128 out of 2434
on 2129 out of 2434
on 2130 out of 2434
on 2131 out of 2434
on 2132 out of 2434
on 2133 out of 2434
on 2134 out of 2434
on 2135 out of 2434
on 2136 out of 2434
on 2137 out of 2434
on 2138 out of 2434
on 2139 out of 2434
on 2140 out of 2434
on 2141 out of 2434
on 2142 out of 2434
on 2143 out of 2434
on 2144 out of 2434
on 2145 out of 2434
on 2146 out of 2434
on 2147 out of 2434
on 2148 out of 2434
on 2149 out of 2434
on 2150 out of 2434
on 2151 out of 2434
on 2152 out of 2434
on 2153 out of 2434
on 2154 out of 2434
on 2155 out of 2434
on 2156 out of 2434
on 2157 out of 2434
on 2158 out of 2434
on 2159 out of 2434
on 2160 out of 2434
on 2161 out of 2434
on 2162 out of 2434
on 2163 out of 2434
on 2164 out of 2434
on 2165 out of 2434
on 2166 out of 2434
on 2167 out of 2434
on 2168 out of 2434
on 2169 out of 2434
on 2170 out of 2434
on 2171 out of 2434
on 2172 out of 2434
on 2173 out of 2434
on 2174 out of 2434
on 2175 out of 2434
on 2176 out of 2434
on 2177 out of 2434


In [168]:
concepts_df.shape
concepts_df.grid.nunique()

(39471, 9)

2393

In [169]:
# add delivery label for each grid 
# this should be the first delivery in the EHR 
concepts_df['delivery_label'] = concepts_df.grid.map(final_labels_dict)
concepts_df['delivery_date'] = concepts_df.grid.map(final_dates_dict)

In [170]:
concepts_df.head(3)

,assertion,cui,grid,ne,pos_in_txt1,pos_in_txt2,semantic,type,delivery_label,delivery_date
0,present,C0700589,R202681100,Contraception,63,76,treatment,NamedEntity,NaN,NaN
1,present,C0332181,R202681100,annual,207,213,test,NamedEntity,NaN,NaN
2,present,C0030350,R205193117,Pap,65,68,test,NamedEntity,NaN,NaN


### Add concept date 

In [171]:
# load key file 
key_df = pd.read_csv(KEY_FILE, sep="\t")

In [172]:
key_df['row_start'] = key_df['row_num'].apply(lambda x: x[2:-2].split(",")[0])
key_df['row_end'] = key_df['row_num'].apply(lambda x: x[2:-2].split(",")[1])
key_df.head()

,GRID,DATE,FORMNAME,FIELDNAME,FIELDVALUE,FIELDLABEL,row_num,row_start,row_end
0,R200006865,2016-01-28,Obstetrics Record,Gu Ta,~ Discussed pain - referred her to PCP\n~ Disc...,Action or Plan,"[(0, 1)]",0,1
1,R200006865,2016-01-28,Obstetrics Record,Cv Ta,Perineum intact\nUterus mobile/NT\nNo adnexal ...,Objective Notes,"[(4, 6)]",4,6
2,R200006865,2016-01-28,Obstetrics Record,Comments Fd,Bleeding has resolved. Breast feeding - notes ...,Subjective Notes,"[(9, 10)]",9,10
3,R200009152,2007-04-26,Obstetrics Record,Gu Ta,iup at 42wks\nBreech\nOligo\n\nSent to ...,Action or Plan,"[(0, 5)]",0,5
4,R200009152,2007-04-26,Obstetrics Record,Cv Ta,see flow,Objective Notes,"[(8, 8)]",8,8


In [173]:
# below: 
# I take the position of concept --> row that concept corresponds to ---> the corresponding clinical note defined by the that row --> get the date for hte clincial note 

In [174]:
concept_w_date_df = pd.DataFrame()
total_grids = concepts_df.grid.nunique()
counter = 0

for this_grid, this_df in concepts_df.groupby('grid'):

    
    counter = counter + 1     
    print("{} of {}".format(counter, total_grids)) if (counter%100 == 0) else None

    
    temp_df = this_df.copy()
    temp_key = key_df.loc[key_df['GRID']==this_grid].copy()
    
    ##
    # row to position in text mapping 
    ##
    
    # load all clinical text file 
    grid_file = os.path.join(RAW_TXT_DIR, "{}-notes.txt".format(this_grid))
    with open(grid_file,'r') as fh: 

        cumul_char_count = 0 # first cursor position is ZERO 

        # each row is one element of this list 
        # each element is the  end position of character 
        end_pos = [] 
        
        for row_num, line in enumerate(fh): 
            char_count = len(line) # subtract one since we start a 0 (3 char (incl trailing space): start 0, end = 2)

            # final cursor position occurs after the space before the next line
            if row_num == 0: 
                start_pos_of_line = cumul_char_count 
            else: 
                start_pos_of_line = cumul_char_count  
            
            end_pos_of_line = start_pos_of_line + (char_count-1) 
            cumul_char_count = cumul_char_count + char_count
            
            end_pos.append(end_pos_of_line)
            
#             print("r{}:s{}-e{} ({})  --> {}".format(row_num, start_pos_of_line, end_pos_of_line, char_count, line))

    end_pos = np.array(end_pos)
    # row number of this concept 
    temp_df['row_num']= temp_df.pos_in_txt1.apply(lambda x: np.min(np.where(int(x)<end_pos)[0]))
#     end_pos
    # map the date to this row number 
    temp_df['date'] = temp_df.row_num.apply(lambda x: temp_key.iloc[np.max(np.where(temp_key.row_start.map(int) <= int(x))[0]),1]) 
    
#     temp_df
    concept_w_date_df = concept_w_date_df.append(temp_df)
    
    if this_grid =="R207060559":
        break

100 of 2393
200 of 2393
300 of 2393
400 of 2393
500 of 2393
600 of 2393
700 of 2393
800 of 2393
900 of 2393
1000 of 2393
1100 of 2393
1200 of 2393
1300 of 2393
1400 of 2393
1500 of 2393
1600 of 2393
1700 of 2393
1800 of 2393
1900 of 2393
2000 of 2393
2100 of 2393
2200 of 2393
2300 of 2393


### format concepts

In [175]:
concept_w_date_df

,assertion,cui,grid,ne,pos_in_txt1,pos_in_txt2,semantic,type,delivery_label,delivery_date,row_num,date
10219,present,C0700589,R200006865,birth control,63,76,treatment,NamedEntity,term,2015-12-19,1,2016-01-28
10220,present,None,R200006865,NT,227,229,problem,NamedEntity,term,2015-12-19,5,2016-01-28
10221,present,None,R200006865,Bleeding,261,269,problem,NamedEntity,term,2015-12-19,9,2016-01-28
10222,present,None,R200006865,reflux,316,322,problem,NamedEntity,term,2015-12-19,9,2016-01-28
10223,present,None,R200006865,an intermittent pain,334,354,problem,NamedEntity,term,2015-12-19,10,2016-01-28
9394,present,None,R200009152,Breech,13,19,problem,NamedEntity,postterm,2007-04-26,1,2007-04-26
9395,present,C0227106,R200009152,primary c,99,108,treatment,NamedEntity,postterm,2007-04-26,4,2007-04-26
9396,present,C0023607,R200009152,LH,111,113,test,NamedEntity,postterm,2007-04-26,5,2007-04-26
5916,present,C0700589,R200009182,Contraception,60,73,treatment,NamedEntity,term,2014-07-13,1,2014-08-30
5917,present,None,R200009182,esp weight gain,170,185,problem,NamedEntity,term,2014-07-13,1,2014-08-30


In [176]:
cgrids = set(concept_w_date_df.grid.unique())
lgrids = set(final_labels_df.GRID.unique())
len(cgrids)
len(lgrids) 

print("number of grids with delivery label: {:,}".format(len(lgrids)))
print("number of grids in concepts_df: {:,}".format(len(cgrids)))
print("Number of grids in concepts_df w/o labels: {:,}".format(len(cgrids.difference(lgrids))))


2370

36379

number of grids with delivery label: 36,379
number of grids in concepts_df: 2,370
Number of grids in concepts_df w/o labels: 483


In [177]:
# remove grids without a label 
concept_w_date_df.shape
concept_w_date_df = concept_w_date_df[~concept_w_date_df.delivery_label.isnull()]

(39061, 12)

In [178]:
# number of hits without a cui 
concept_w_date_df[concept_w_date_df.cui.isnull()].shape[0]
# number of grids without a cui 
concept_w_date_df[concept_w_date_df.cui.isnull()].grid.nunique()

13905

1565

In [179]:
concept_w_date_df.shape
concept_w_date_df.grid.nunique()
concept_w_date_df.head()

(32241, 12)

1887

,assertion,cui,grid,ne,pos_in_txt1,pos_in_txt2,semantic,type,delivery_label,delivery_date,row_num,date
10219,present,C0700589,R200006865,birth control,63,76,treatment,NamedEntity,term,2015-12-19,1,2016-01-28
10220,present,None,R200006865,NT,227,229,problem,NamedEntity,term,2015-12-19,5,2016-01-28
10221,present,None,R200006865,Bleeding,261,269,problem,NamedEntity,term,2015-12-19,9,2016-01-28
10222,present,None,R200006865,reflux,316,322,problem,NamedEntity,term,2015-12-19,9,2016-01-28
10223,present,None,R200006865,an intermittent pain,334,354,problem,NamedEntity,term,2015-12-19,10,2016-01-28


### keep concepts acquired only for specified time frame 

In [186]:
concept_w_date_df['date'] = pd.to_datetime(concept_w_date_df['date'])
concept_w_date_df['delivery_date'] = pd.to_datetime(concept_w_date_df['delivery_date'])

/Users/abraha1/anaconda3/envs/verBio/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/abraha1/anaconda3/envs/verBio/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [188]:
# positive if date occured after delivery date 
concept_w_date_df['days_diff'] = concept_w_date_df['date'] - concept_w_date_df['delivery_date']

/Users/abraha1/anaconda3/envs/verBio/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [189]:
concept_w_date_df.head()

,assertion,cui,grid,ne,pos_in_txt1,pos_in_txt2,semantic,type,delivery_label,delivery_date,row_num,date,days_diff
10219,present,C0700589,R200006865,birth control,63,76,treatment,NamedEntity,term,2015-12-19,1,2016-01-28,40 days
10220,present,None,R200006865,NT,227,229,problem,NamedEntity,term,2015-12-19,5,2016-01-28,40 days
10221,present,None,R200006865,Bleeding,261,269,problem,NamedEntity,term,2015-12-19,9,2016-01-28,40 days
10222,present,None,R200006865,reflux,316,322,problem,NamedEntity,term,2015-12-19,9,2016-01-28,40 days
10223,present,None,R200006865,an intermittent pain,334,354,problem,NamedEntity,term,2015-12-19,10,2016-01-28,40 days


In [193]:
# keep only data occuring 90 days before delivery date 
# keep days_diff < -90 

filtered_concept_df = concept_w_date_df.loc[concept_w_date_df['days_diff'] < np.timedelta64(-90,'D')].copy()

In [195]:

concept_w_date_df.shape[0]
filtered_concept_df.shape[0]

concept_w_date_df.grid.nunique()
filtered_concept_df.grid.nunique()

32241

3490

1887

145

## Extract CUI

In [196]:
cui_df = filtered_concept_df[~filtered_concept_df.cui.isnull()].copy()

In [201]:
cui_df.head()
cui_df.shape
cui_df.grid.nunique()

,assertion,cui,grid,ne,pos_in_txt1,pos_in_txt2,semantic,type,delivery_label,delivery_date,row_num,date,days_diff
1560,present,C2963416,R200063476,Prenatal panel,637,651,test,NamedEntity,term,2016-11-12,18,2016-03-30,-227 days
1561,present,C1610733,R200063476,Urine specimen,670,684,test,NamedEntity,term,2016-11-12,19,2016-03-30,-227 days
1562,present,C1858668,R200063476,GC/CT,699,704,test,NamedEntity,term,2016-11-12,19,2016-03-30,-227 days
1563,present,C0107676,R200063476,C&S,709,712,test,NamedEntity,term,2016-11-12,19,2016-03-30,-227 days
1564,present,C1659567,R200063476,Thin Prep pap,715,728,test,NamedEntity,term,2016-11-12,20,2016-03-30,-227 days


(1818, 13)

138

In [202]:
long_cui_df = concepts_df.groupby(['grid','cui']).size().reset_index().rename(columns={0:'counts'})
wide_cui_df = long_cui_df.pivot(index='grid',columns='cui', values='counts').fillna(0)

In [203]:
wide_cui_df.head()
wide_cui_df.shape
# number of grids w/ at least 1 instance of this cui 
wide_cui_df.apply(lambda x: x > 0).sum().sort_values(ascending=False).head()


cui,C0000937,C0001465,C0001899,C0002210,C0002461,C0002627,C0002766,C0002771,C0002808,C0002903,...,C3854296,C3854689,C3865400,C3870349,C3871278,C3871284,C3872403,C3877551,C3877576,C3877849
grid,,,,,,,,,,,,,,,,,,,,,
R200006865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R200009152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R200009182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R200020899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R200021053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(2322, 1976)

cui
C0700589    1141
C0748691     600
C0332181     597
C0030350     483
C0018378     365
dtype: int64

In [204]:
cui_freq_thresh  = wide_cui_df.shape[0] * 0.5/100
print(cui_freq_thresh)

# number of cui present after filterering on frequency 
selected_cui_df = wide_cui_df.apply(lambda x: x > 0).sum()[wide_cui_df.apply(lambda x: x > 0).sum() > cui_freq_thresh].reset_index().rename(columns={0:'count'})

11.61


In [207]:
selected_cui_df.head()
selected_cui_df.shape

,cui,count
0,C0002210,16
1,C0002808,134
2,C0003232,33
3,C0004683,26
4,C0004732,24


(203, 2)

## Data distribution for concepts

In [29]:
# number of unique concepts 
concepts_df['ne'].nunique()

7246

In [30]:
# distribution of number of concepts per person 
sns.set(style='whitegrid',rc={'figure.figsize':(12,6)}, font_scale=1.2)
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
concepts_df.groupby('grid').count()['type'].plot.hist(bins=50, ax=ax1)

# distribution of  number of unique concepts per person
concepts_df.groupby(['grid','ne']).count().reset_index().groupby('grid').count()['ne'].plot.hist(bins=50, ax = ax2)

# Feature Selection 

In [18]:
# check for any missing values in 'ne' column
concepts_df[concepts_df['ne'].isnull()]

,assertion,cui,grid,ne,pos_in_txt1,pos_in_txt2,semantic,type,delivery_label


## CUIs

In [208]:
cui_feat_df = wide_cui_df.loc[:, selected_cui_df.cui].copy()
cui_feat_df.reset_index(inplace=True)

In [210]:
#write 
# cui_feat_file = os.path.join(root_path, "/abraha1/projects/PTB_phenotyping/data/ptb_predict_machine_learning/feature_matrices/ob_notes_unstruct", "filtered_cui_counts_ob_notes_feat_mat.tsv")
cui_feat_file = os.path.join(root_path, "abraha1/projects/PTB_phenotyping/data/ptb_predict_machine_learning/feature_matrices/ob_notes_unstruct", "filtered_cui_counts_ob_notes_feat_mat_up_to_90days_before_delivery.tsv")
cui_feat_df.to_csv(cui_feat_file, sep="\t", index=False)

In [212]:
cui_feat_df.head()

cui,grid,C0002210,C0002808,C0003232,C0004683,C0004732,C0005802,C0005823,C0005824,C0005841,...,C2698624,C2708334,C2730299,C2985321,C2986775,C3644171,C3840199,C3841297,C3842800,C3845665
0,R200006865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,R200009152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,R200009182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,R200020899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,R200021053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Concepts

### clean up concepts (upper/lower case)

In [19]:
lower_cols = [x.lower() for x in concepts_df['ne'].unique()]
col_df = pd.DataFrame({'COLNAME':concepts_df['ne'].unique(), 'colname':lower_cols})

In [20]:
def create_set(x):
    return set(x.COLNAME.values)

In [21]:
col_df.groupby('colname').apply(lambda x : create_set(x) ).reset_index().rename(columns={0:''})

,colname,
0,"""cysts"" in her groin area","{""cysts"" in her groin area}"
1,"""low fluid","{""low fluid}"
2,1 cm above pubic symphysis,{1 cm above pubic symphysis}
3,1 cm nontender mass right axilla,{1 cm nontender mass right axilla}
4,1 x 2cm red fluid filled lesion,{1 X 2cm red fluid filled lesion}
5,1% lidocaine,{1% lidocaine}
6,1+ ankle edema,{1+ ankle edema}
7,1+ le edema,{1+ LE edema}
8,1+ pitting edema,{1+ pitting edema}
9,1+le edema,{1+LE edema}


### create feature matrix

In [139]:
long_counts_df = concepts_df.groupby(['grid','ne']).size().reset_index().rename(columns={0:'counts'})
raw_matrix_df = long_counts_df.pivot(index='grid',columns='ne', values='counts').fillna(0)

In [140]:
# create frequency of hits for each features
concept_counts = raw_matrix_df.sum(0).sort_values(ascending=False).reset_index().rename(columns={0:'count'})
concept_counts.head()

,ne,count
0,Contraception,1438.0
1,contraception,831.0
2,detailed exam,578.0
3,annual,576.0
4,GBS,489.0


### Remove features with low frequency 

In [141]:
# retain only feature that have freq > 1% of number of grids 
freq_thresh  = raw_matrix_df.shape[0] * 0.5/100
print("Min number of counts requried: {}".format(freq_thresh))
feat_to_retain = concept_counts.loc[concept_counts['count'] > freq_thresh, 'ne'].values
feat_mat_df = raw_matrix_df.loc[:, feat_to_retain].reset_index()

Min number of counts requried: 9.515


In [142]:
# add delivery labels
feat_mat_df['label'] = feat_mat_df.grid.map(final_labels_dict)

In [143]:
feat_mat_df.head(3)

ne,grid,Contraception,contraception,detailed exam,annual,GBS,US,DMS,PE,Pap,...,incision check,Back pain,FAVD,Postpartum exam,GU fxn,BPPs,steri-strips,weight changes,pyelo,label
0,R200006865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,term
1,R200009152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,postterm
2,R200009182,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,term


In [160]:
feat_mat_df.shape

(1903, 479)

### write feat matrix

In [211]:
!$FEAT_OUTPUT_DIR

/bin/sh: /Volumes/abraha1/projects/PTB_phenotyping/data/ptb_predict_machine_learning/feature_matrices/ob_notes_unstruct: is a directory


In [167]:
feat_mat_df.head()
feat_file=os.path.join(FEAT_OUTPUT_DIR, 'all_concepts_raw_count_feat_mat.tsv')
feat_mat_df.rename(columns={'grid':'GRID'}, inplace=True)
feat_mat_df.drop('label',axis=1).to_csv(feat_file, sep="\t", index=False)

ne,grid,Contraception,contraception,detailed exam,annual,GBS,US,DMS,PE,Pap,...,incision check,Back pain,FAVD,Postpartum exam,GU fxn,BPPs,steri-strips,weight changes,pyelo,label
0,R200006865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,term
1,R200009152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,postterm
2,R200009182,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,term
3,R200020899,1.0,0.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,term
4,R200021053,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,preterm


# Predictive preliminary models

In [151]:
from xgboost.sklearn import XGBClassifier

In [146]:
X_mat = feat_mat_df.drop(['grid','label'],axis=1).values
y_label = feat_mat_df.label.apply(lambda x: 1 if x == 'preterm' else 0).values

In [148]:
X_train, y_train, X_test, y_test, annotated_df =  create_held_out(X_mat, y_label, feat_mat_df)

In [149]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [154]:
sk_params = {'n_estimators':10,
             'max_depth': 4,
             'learning_rate':0.1,
             'random_state': 32,
             'objective':'binary:logistic',
             'scale_pos_weight': float(np.sum(y_train == 0)) / np.sum(y_train == 1), 
             'importance_type':'gain',
             'silent': 1}

In [155]:
best_xgb_rf = XGBClassifier(**sk_params)
best_xgb_rf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=32, reg_alpha=0, reg_lambda=1,
       scale_pos_weight=5.341666666666667, seed=None, silent=1,
       subsample=1)

In [157]:
metrics_results, metrics_df, model_params = validate_best_model(best_xgb_rf, X_test, y_test)

Validating best model on held out set...


/scratch/abraha1/conda_scratch/envs/verbioR/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [158]:
metrics_df.head()

,precision,recall,f1,roc_auc,avg_pr,tn_count,fp_count,fn_count,tp_count,brier_score,total_count
0,0.283333,0.283333,0.283333,0.661812,0.250232,278,43,43,17,0.232256,381
